## Template Matching

In [1]:
import cv2

method = cv2.TM_SQDIFF_NORMED

# Read the images from the file

small_image = cv2.imread('destijl_dataset/02_image/0006.png')
large_image = cv2.imread('destijl_dataset/00_preview_png/0006.png')

result = cv2.matchTemplate(small_image, large_image, method)

# We want the minimum squared difference
mn,_,mnLoc,_ = cv2.minMaxLoc(result)

# Draw the rectangle:
# Extract the coordinates of our best match
MPx,MPy = mnLoc

# Step 2: Get the size of the template. This is the same size as the match.
trows,tcols = small_image.shape[:2]

# Step 3: Draw the rectangle on large_image
cv2.rectangle(large_image, (MPx,MPy),(MPx+tcols,MPy+trows),(0,0,255),2)

# Display the original image with the rectangle around the match.
cv2.imshow('output',large_image)

# The image is only displayed if we call this
cv2.waitKey(0)

## Contour

In [10]:
import numpy as np
import cv2

image = cv2.imread("destijl_dataset/03_decoration/0346.png")
dimensions = image.shape

height= image.shape[0]
width = image.shape[1]
size = height*width

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)


_,thresh = cv2.threshold(gray,150,255,cv2.THRESH_BINARY_INV)

cnts, hier = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
size_elements = 0
for cnt in cnts:
    cv2.drawContours(image,cnts, -1, (0, 255, 0), 3)
    size_elements += cv2.contourArea(cnt)
    print(cv2.contourArea(cnt))



cv2.imshow("Image", image)
print("size elements total : ", size_elements)
print("size of pic : ", size)
print("rate of fullness : % ", (size_elements/size)*100)
cv2.waitKey(0)

3215.0
8003.5
5967.0
12517.5
11133.5
3327.0
6261.0
3233.5
24508.0
8146.0
6438.5
6467.0
6421.0
1389.5
3477.0
2578.5
5462.5
4832.5
1414.0
2706.5
1379.5
size elements total :  128878.5
size of pic :  12169548
rate of fullness : %  1.0590245422426536


## Get Colors from KMeans then mask each color

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans
from matplotlib import pyplot as plt
import os
import imageio.v3 as iio
from PIL import Image
from colormap import rgb2hex
from matplotlib.colors import ListedColormap
import colorsys
import cv2
from PIL import Image

########################### PARAMETERS ###########################
layers = [
    '02_image',
    '03_decoration',
    '04_text'
]
selected_layer = 0

image_path = "destijl_dataset/03_decoration/0346.png"
# read image
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
assert img is not None, "file could not be read, check with os.path.exists()"
img = cv2.medianBlur(img,5)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
 cv2.THRESH_BINARY,11,2)
th3 = cv2.dilate(th3, None, iterations = 1)

# view result

result = img.copy()
contours = cv2.findContours(th3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
contours = contours[0] if len(contours) == 2 else contours[1]
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(result, (x, y), (x+w-1, y+h-1), (0, 0, 255), 1)

# plt.imshow(result,'gray')
# plt.imshow(th3,'gray')
# view result
# cv2.imshow("threshold", th3)
cv2.imwrite("white_green_threshold.jpg", th3)
cv2.imwrite("white_green_result.jpg", result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


True

## Canny Edge Detector for extracting design elements

In [63]:
image_path = "destijl_dataset/03_decoration/0346.png"
# read image
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
img_color = cv2.imread(image_path)
img_color2 = img_color.copy()
assert img is not None, "file could not be read, check with os.path.exists()"

image_edges = cv2.Canny(img,100,200)
cv2.imwrite("canny.jpg", image_edges)
contours, hierarchy = cv2.findContours(image_edges, 
    cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
#cv2.dilate(img_color, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1), (1, 1)))
cv2.drawContours(img_color, contours, -1, (0, 255, 0), 3)
cv2.imwrite("canny_contour.jpg", img_color)

colors = []
# For each list of contour points...
for c in contours:
    M = cv2.moments(c)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        colors.append(img_color2[cY, cX])

for cnt in contours:
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img_color2,[box],0,(0,0,255),2)
cv2.imwrite("canny_contour.jpg", img_color2)



True

In [34]:
len(colors)

43

In [48]:
rectangles = []
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    if (x,y,w,h) not in rectangles:
        rectangles.append((x,y,w,h))

for rect in rectangles:
    x, y, w, h = rect
    cv2.rectangle(img_color2, (x, y), (x+w-1, y+h-1), (0, 0, 255), 2)

#Reverting the original image back to BGR so we can draw in colors
#parameter -1 specifies that we want to draw all the contours
cv2.imwrite("canny_contour.jpg", img_color2)

True

In [186]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib.colors import hsv_to_rgb, rgb_to_hsv


def determine_num_colors(image_path, min_colors, max_colors):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to the RGB color space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Reshape the image to a 2D array of pixels
    pixels = image.reshape(-1, 3)
    
    # Initialize variables for storing optimal number of colors and silhouette score
    best_num_colors = 0
    best_score = -1
    
    # Iterate through the range of possible number of colors
    for num_colors in range(min_colors, max_colors + 1):
        # Apply K-means clustering with the specified number of colors
        kmeans = KMeans(n_clusters=num_colors)
        kmeans.fit(pixels)
        
        # Get the labels for the pixels
        labels = kmeans.labels_
        
        # Calculate the silhouette score
        score = silhouette_score(pixels, labels)
        
        # Update the optimal number of colors if the current score is higher
        if score > best_score:
            best_score = score
            best_num_colors = num_colors
    
    return best_num_colors

def separate_colors(image_path):
    # Determine the number of dominant colors
    min_colors = 2
    max_colors = 5
    #num_colors = determine_num_colors(image_path, min_colors, max_colors)
    num_colors = 6
    
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to the RGB color space
    image3 = image.copy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image2 = image.copy()
    
    # Reshape the image to a 2D array of pixels
    pixels = image.reshape(-1, 3)
    
    # Apply K-means clustering with the determined number of colors
    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(pixels)
    
    # Get the RGB values of the dominant colors
    colors = kmeans.cluster_centers_.astype(int)
    print("Num of colors: ", len(colors))
    
    # Convert the colors to the HSV color space
    hsv_colors = [] 

    for i, color in enumerate(colors):
            x, y, z = color
            if not (252 < x < 256 and 252 < y < 256 and 252 < z < 256):
                x, y, z = rgb_to_hsv([x/255, y/255, z/255])
                hsv_colors.append([x*255, y*255, z*255])
    print("Num of selected colors: ", len(hsv_colors))
    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image2, cv2.COLOR_RGB2HSV)
    
    # Create masks for each dominant color
    masks = []
    hsv_colors = np.asarray(hsv_colors, dtype=np.int32)
    
    for i in range(len(hsv_colors)):
        
        h, s, v = hsv_colors[i, :]
        lower_color = hsv_colors[i, :] - np.array([5, 50+s, 50+v])
        upper_color = hsv_colors[i, :] + np.array([15, 255, 255])
        mask = cv2.inRange(hsv_image, lower_color, upper_color)
        masks.append(mask)
    
    # Find contours in each mask
    contours = []
    for mask in masks:
        contours_color, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contours.append(contours_color)
    
    # Draw bounding boxes around the shapes
    print("bounding boxess")
    image_with_boxes = image.copy()
    for i, contour_color in enumerate(contours):
        for contour in contour_color:
            print("is there any rects??")
            x, y, w, h = cv2.boundingRect(contour)
            print(x, y, w, h)
            cv2.rectangle(image_with_boxes, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Display the resulting image
    cv2.imwrite('contour_thingy.jpg', image_with_boxes)

# Example usage
image_path = 'destijl_dataset/03_decoration/0663.png'  # Replace 'your_image.jpg' with the path to your image

separate_colors(image_path)

Num of colors:  6
Num of selected colors:  5
bounding boxess
